In [ ]:
#author_Hamid_Fsian
#Emotion Detection using Faces
#this script is basically to load my database and train my model,
#the i will upload my model into my machine in order to use OpenCV and make it
#run in real Time

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2 as cv
#ML libraries
import tensorflow as tf
from tensorflow import keras 
from keras.preprocessing.image import load_img, img_to_array #to load image and make them an array
from keras.preprocessing.image import ImageDataGenerator #generate image from a path
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
#from keras.optimizers import Adam,SGD,RMSprop

In [ ]:
picture_size = 48 #standard our pic to 48by48


In [ ]:
picture_size = 48
path_train = "drive/MyDrive/FaceEmotion/NewFolder/train/" #the path for my training data
path_validation = "drive/MyDrive/FaceEmotion/NewFolder/validation/" #path for my validation data

In [ ]:
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
from skimage import io
#this code is here just to check if my iamges are loaded correctly
path = "drive/MyDrive/FaceEmotion/NewFolder/train/angry/0.jpg"
image = io.imread(path,plugin='matplotlib')
image = np.array(image, dtype=np.uint8)
print(image.shape)
#gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
cv2_imshow(image)


In [ ]:

batch_size  = 32 #number of training exemples that my model will see in 1 iteration
#2^n better use this type of batch size
datagen_train  = ImageDataGenerator() #generate image for the training part
datagen_val = ImageDataGenerator() #same thing for validation data

#train_set va contenir le datagen_train qui lui à travers(flow_from_directory) ira chercher ce dossier du path train
#greyscale parce que c'est en lvl grey
#categorical because i have 7 classes
#shuffle pour melanger
#NB : we don't need to shuffle the validation pack
train_set = datagen_train.flow_from_directory(path_train,
                                              target_size = (picture_size,picture_size),
                                              color_mode = "grayscale",
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=True)
test_set = datagen_val.flow_from_directory(path_validation,
                                              target_size = (picture_size,picture_size),
                                              color_mode = "grayscale",
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=False)
#Found 28831 images belonging to 7 classes.
#Found 7066 images belonging to 7 classes.

In [ ]:
 ## Building the network
no_of_classes = 7 #because i have 7 classes

model = Sequential() 

#1st CNN layer
model.add(Conv2D(32,(3,3),padding = 'same',input_shape = (48,48,1))) #nombre de filtres, apres taille 48,48,1 psk c'est notre taille 1psk greyscale
model.add(BatchNormalization()) #normaliser mes data automatiquement
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))#desactiver aleatoirement des neuronnes(25%) pour eviter que mon modéle ne soit dependant de mes data base

#2nd CNN layer
model.add(Conv2D(64,(5,5),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#3rd CNN layer
model.add(Conv2D(128,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.25))

#4th CNN layer
model.add(Conv2D(128,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten()) #pour les mettres dans un vecteur


#Fully connected Layers
#FC 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))


# FC layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(no_of_classes, activation='softmax'))


opt = keras.optimizers.Adam(learning_rate=0.0001) 

model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy']) #CatCross... parceque j'ai plusieurs classes 
model.summary()

In [ ]:
#most intresting part
#EarlyStopping : in order to stop my model from computing if the accuracy stagnate. and avoid overfiting
#ReduceLRonPlateau : in order to reduce the learning rate when one iteration is complete and become quicker

from keras.callbacks import ModelCheckpoint , EarlyStopping , ReduceLROnPlateau 
checkpoint = ModelCheckpoint("./modelProf.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#modelchekpoint is used pour enregistrer ton model en ou paramétres dans un chekpoint file (generalement avec model.fit)
early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )#si ma precision n'augmente pas, alors on arréte et on évite le overfitting

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

epochs = 48
optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss='categorical_crossentropy',
              optimizer = optimizer,
              metrics=['accuracy'])

In [ ]:

history = model.fit_generator(generator=train_set,
                                steps_per_epoch=train_set.n//train_set.batch_size,
                                epochs=epochs,
                                validation_data = test_set,
                                validation_steps = test_set.n//test_set.batch_size,
                                callbacks=callbacks_list
                                )